In [1]:
import pandas as pd
from antarctic.pandas_fields import SeriesField, FrameField

In [2]:
# Connect to a MongoDB database
from mongoengine import *
client = connect(db="test", host="mongomock://localhost")

In [3]:
# Read some random prices
p = pd.read_csv("data/price.csv", squeeze=True, index_col=0)
p

,A,B,C,D,E,F,G
2013-01-01,1673.78,23311.98,62550.10,3735.12,1462.42,2711.25,2518.99
2013-01-02,1686.90,23311.98,62550.10,3735.12,1462.42,2711.25,2518.99
2013-01-03,1663.95,23398.60,63312.46,3714.99,1459.37,2701.22,2509.51
2013-01-04,1655.65,23331.09,62523.06,3689.34,1466.47,2709.35,2516.81
2013-01-07,1646.95,23329.75,61932.54,3699.14,1461.89,2695.56,2523.77
...,...,...,...,...,...,...,...
2015-04-16,1198.56,27739.71,54674.21,2694.50,2104.99,3751.72,3077.21
2015-04-17,1204.27,27653.12,53954.79,2676.61,2081.18,3674.05,3061.38
2015-04-20,1195.88,27094.93,53761.27,2667.63,2100.40,3718.04,3059.73
2015-04-21,1202.34,27850.49,NaN,2651.41,2097.29,3719.38,3065.53


## Pandas objects within the standard Documents

In [4]:
# A first option could be to store the entire frame as one field in a Document
class Portfolio(Document):
    prices = FrameField()
    series = SeriesField()

In [5]:
# create a Portfolio object and store in a database
portfolio = Portfolio(prices=p, series=p["A"]).save()

In [6]:
portfolio.prices

,A,B,C,D,E,F,G
2013-01-01,1673.78,23311.98,62550.10,3735.12,1462.42,2711.25,2518.99
2013-01-02,1686.90,23311.98,62550.10,3735.12,1462.42,2711.25,2518.99
2013-01-03,1663.95,23398.60,63312.46,3714.99,1459.37,2701.22,2509.51
2013-01-04,1655.65,23331.09,62523.06,3689.34,1466.47,2709.35,2516.81
2013-01-07,1646.95,23329.75,61932.54,3699.14,1461.89,2695.56,2523.77
...,...,...,...,...,...,...,...
2015-04-16,1198.56,27739.71,54674.21,2694.50,2104.99,3751.72,3077.21
2015-04-17,1204.27,27653.12,53954.79,2676.61,2081.18,3674.05,3061.38
2015-04-20,1195.88,27094.93,53761.27,2667.63,2100.40,3718.04,3059.73
2015-04-21,1202.34,27850.49,NaN,2651.41,2097.29,3719.38,3065.53


In [7]:
portfolio.series

2013-01-01    1673.78
2013-01-02    1686.90
2013-01-03    1663.95
2013-01-04    1655.65
2013-01-07    1646.95
               ...   
2015-04-16    1198.56
2015-04-17    1204.27
2015-04-20    1195.88
2015-04-21    1202.34
2015-04-22    1200.59
Name: A, Length: 602, dtype: float64

## Using the XDocument provided by antarctic

The XDocument is a convenient choice for storing a group of Documents of the same type

In [8]:
from antarctic.document import XDocument

In [9]:
class Symbol(XDocument):
    price = SeriesField()
    volume = SeriesField()

Symbol.objects.delete()

# XDocuments have the field name and the field reference
for name, data in p.items():
    s = Symbol(name=name, price=data)
    s.reference["Full name"] = name + " full"
    s.save()

In [10]:
# each Symbol is an XDocument. The class XDocument provides a few convenient cls methods
Symbol.reference_frame()

,Full name
symbol,
A,A full
B,B full
C,C full
D,D full
E,E full
F,F full
G,G full


In [11]:
# you can ask for subsets
symbols=Symbol.subset(names=["A","B","D"])

# or all of them
symbols_all=Symbol.subset()


In [12]:
symbols

[<Symbol: A>, <Symbol: B>, <Symbol: D>]

In [14]:
d = Symbol.to_dict()
d

{'A': <Symbol: A>,
 'B': <Symbol: B>,
 'C': <Symbol: C>,
 'D': <Symbol: D>,
 'E': <Symbol: E>,
 'F': <Symbol: F>,
 'G': <Symbol: G>}

In [15]:
d = Symbol.to_dict(objects=symbols)
d

{'A': <Symbol: A>, 'B': <Symbol: B>, 'D': <Symbol: D>}

In [ ]:
Symbol.frame("price",objects=symbols)

In [ ]:
import numpy as np
# Do same fancy stuff and apply functions to (subsets of) XDocuments
for name, result in Symbol.apply(func=lambda s: s.price.mean(), default=np.nan, objects=symbols):
    print(name, result)

,A,B,D
2013-01-01,1673.78,23311.98,3735.12
2013-01-02,1686.90,23311.98,3735.12
2013-01-03,1663.95,23398.60,3714.99
2013-01-04,1655.65,23331.09,3689.34
2013-01-07,1646.95,23329.75,3699.14
...,...,...,...
2015-04-16,1198.56,27739.71,2694.50
2015-04-17,1204.27,27653.12,2676.61
2015-04-20,1195.88,27094.93,2667.63
2015-04-21,1202.34,27850.49,2651.41


In [22]:
import numpy as np
# Do same fancy stuff and apply functions to (subsets of) XDocuments
for name, result in Symbol.apply(func=lambda s: s.price.mean(), default=np.nan, objects=symbols):
    print(name, result)

A 1322.0910132890367
B 23171.895
D 3422.3647848537003
